In [45]:
#from tensorflow_docs.vis import embed
from tensorflow import keras
#from imutils import paths

#import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
#import imageio
import cv2
import os
import json

Data preparation

Preparing training data

In [46]:
# data_df = pd. read_csv('/home/app/src/data/CSV/dataset_df.csv', dtype={"shot": str})

# data_df.drop(columns=['scale_val'], inplace=True)
# data_df.drop(columns = ['move_val'], inplace=True)

# train_path = '/workspaces/final-project-shot-type/data/shot-type-dataset/trailer_v3/train/'
# data_df['video_name'] = train_path + 'shot_' + data_df['movie'] + '_' + data_df['shot'] + '.mp4'
# train_df = data_df[(data_df['dataset']=='train') | (data_df['dataset']=='val') ]
# train_df = train_df[['video_name', 'move_label','scale_label']]
# train_df = train_df.rename(columns ={'scale_label': 'tag'})


# test_path = '/workspaces/final-project-shot-type/data/shot-type-dataset/trailer_v3/test/'
# data_df['video_name'] = test_path + 'shot_' + data_df['movie'] + '_' + data_df['shot'] + '.mp4'
# test_df = data_df[data_df['dataset']=='test']
# test_df = test_df[['video_name', 'move_label','scale_label']]
# test_df = test_df.rename(columns ={'scale_label': 'tag'})

In [47]:
#train_df = pd.read_csv('/home/app/src/data/CSV/dataset_train_10k_v1.csv', )

In [48]:
#test_df = pd.read_csv('/home/app/src/data/CSV/dataset_test_10k_v1.csv', )

In [49]:
#df_train.to_csv('/workspaces/final-project-shot-type/data/CSV/dataset_train_v1.csv')

Preparing Test Data

In [50]:
#df_test.to_csv('/workspaces/final-project-shot-type/data/CSV/dataset_test_v1.csv')

In [51]:
train_df= pd.read_csv('/home/app/src/data/CSV/dataset_train_v2+v3.csv', )
test_df= pd.read_csv('/home/app/src/data/CSV/dataset_test_v3.csv', )

In [52]:
train_df= train_df

# train_df = pd.read_csv('/workspaces/final-project-shot-type/data/CSV/dataset_train_v1.csv')
test_df = test_df

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 26072
Total videos for testing: 8187


,video_name,move_label,tag
13202,/home/app/src/data/shot-type-dataset/trailer_v...,Motion,MS
2185,/home/app/src/data/shot-type-dataset/trailer_v...,Static,MS
4442,/home/app/src/data/shot-type-dataset/trailer_v...,Pull,MS
9549,/home/app/src/data/shot-type-dataset/trailer_v...,Motion,MS
181,/home/app/src/data/shot-type-dataset/trailer_v...,Motion,LS
21663,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS
4631,/home/app/src/data/shot-type-dataset/trailer_v...,Static,ECS
15277,/home/app/src/data/shot-type-dataset/trailer_v...,Static,CS
18905,/home/app/src/data/shot-type-dataset/trailer_v...,Static,ECS
17812,/home/app/src/data/shot-type-dataset/trailer_v...,Pull,LS


In [53]:
train_df['tag'].value_counts()

MS      5656
CS      5578
FS      5161
ECS     4868
LS      4203
None     606
Name: tag, dtype: int64

Define hyperparameters

In [54]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 25

MAX_SEQ_LENGTH = 10
NUM_FEATURES = 2048

Loading frames and working on frame's size

In [55]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

Feature extractor

In [56]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()
print(feature_extractor.summary())

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_2 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________________________________________
None


Working on labels

In [57]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['CS', 'ECS', 'FS', 'LS', 'MS', 'None']


Preparing videos

In [58]:
# def prepare_all_videos(df, root_dir):
#     num_samples = len(df)
#     video_paths = df["video_name"].values.tolist()
#     labels = df["tag"].values
#     labels = label_processor(labels[..., None]).numpy()

#     # `frame_masks` and `frame_features` are what we will feed to our sequence model.
#     # `frame_masks` will contain a bunch of booleans denoting if a timestep is
#     # masked with padding or not.
#     frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
#     frame_features = np.zeros(
#         shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
#     )

#     # For each video.
#     for idx, path in enumerate(video_paths):
#         # Gather all its frames and add a batch dimension.
#         frames = load_video(os.path.join(root_dir, path))
#         frames = frames[None, ...]

#         # Initialize placeholders to store the masks and features of the current video.
#         temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
#         temp_frame_features = np.zeros(
#             shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
#         )

#         # Extract features from the frames of the current video.
#         for i, batch in enumerate(frames):
#             video_length = batch.shape[0]
#             length = min(MAX_SEQ_LENGTH, video_length)
#             for j in range(length):
#                 temp_frame_features[i, j, :] = feature_extractor.predict(
#                     batch[None, j, :]
#                 )
#             temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

#         frame_features[idx,] = temp_frame_features.squeeze()
#         frame_masks[idx,] = temp_frame_mask.squeeze()

#     return (frame_features, frame_masks), labels


# train_data, train_labels = prepare_all_videos(train_df, "train")
# test_data, test_labels = prepare_all_videos(test_df, "test")

# print(f"Frame features in train set: {train_data[0].shape}")
# print(f"Frame masks in train set: {train_data[1].shape}")

Saving embeddings

Train

In [59]:
#train_data_embedding_0_10k = np.save('/workspaces/final-project-shot-type/embeddings/train_data_embedding_0_10k.npy', train_data[0])

In [60]:
#train_data_embedding_1_10k = np.save('/workspaces/final-project-shot-type/embeddings/train_data_embedding_1_10k.npy', train_data[1])

In [61]:
#train_labels_embedding_10k = np.save('/workspaces/final-project-shot-type/embeddings/train_labels_embedding_10k.npy', train_labels)

Test

In [62]:
#test_data_embedding_0_10k = np.save('/workspaces/final-project-shot-type/embeddings/test_data_embedding_0_10k.npy', test_data[0])

In [63]:
#test_data_embedding_1_10k = np.save('/workspaces/final-project-shot-type/embeddings/test_data_embedding_1_10k.npy', test_data[1])

In [64]:
#test_labels_embedding_10k = np.save('/workspaces/final-project-shot-type/embeddings/test_labels_embedding_10k.npy', test_labels)

Loading embeddings

In [65]:
path_load_embeddings= '/home/app/src/embeddings/static_shot/10k_embeddings/'

#Train
train_data_0 = np.load(path_load_embeddings + 'train_data_embedding_0_10k.npy')
train_data_1 = np.load(path_load_embeddings + 'train_data_embedding_1_10k.npy')
train_data=(train_data_0, train_data_1)
train_labels = np.load(path_load_embeddings + 'train_labels_embedding_10k.npy')

#Test
test_data_0 = np.load(path_load_embeddings + 'test_data_embedding_0_10k.npy')
test_data_1 = np.load(path_load_embeddings + 'test_data_embedding_1_10k.npy')
test_data=(test_data_0, test_data_1)
test_labels = np.load(path_load_embeddings + 'test_labels_embedding_10k.npy')

Sequence model (RNN)

In [66]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"],
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights_exp_005.h5"
    filepath_log = '/home/app/src/experiments/inceptionv3_GRU/logs/static'
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.000001)
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )
    tensor_board = keras.callbacks.TensorBoard (filepath_log)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint, reduce_lr, tensor_board],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/25
219/219 [==============================] - ETA: 0s - loss: 1.6376 - accuracy: 0.2859
Epoch 1: val_loss improved from inf to 1.50818, saving model to /home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights_exp_005.h5
219/219 [==============================] - 15s 31ms/step - loss: 1.6376 - accuracy: 0.2859 - val_loss: 1.5082 - val_accuracy: 0.4280 - lr: 1.0000e-04
Epoch 2/25
216/219 [============================>.] - ETA: 0s - loss: 1.4658 - accuracy: 0.3885
Epoch 2: val_loss improved from 1.50818 to 1.33487, saving model to /home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights_exp_005.h5
219/219 [==============================] - 4s 17ms/step - loss: 1.4661 - accuracy: 0.3886 - val_loss: 1.3349 - val_accuracy: 0.4763 - lr: 1.0000e-04
Epoch 3/25
219/219 [==============================] - ETA: 0s - loss: 1.3445 - accuracy: 0.4443
Epoch 3: val_loss improved from 1.33487 to 1.21826, saving model to /home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights

Predictions

In [71]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# def to_gif(images):
#     converted_images = images.astype(np.uint8)
#     imageio.mimsave("animation.gif", converted_images, fps=10)
#     return embed.embed_file("animation.gif")


test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
# to_gif(test_frames[:MAX_SEQ_LENGTH])

Test video path: /home/app/src/data/shot-type-dataset/trailer_v3/test/shot_tt3404234_0008.mp4
  FS: 82.54%
  LS:  9.14%
  ECS:  6.10%
  CS:  1.08%
  MS:  1.05%
  None:  0.09%
